# Welcome to Kluster, a distributed multibeam processing framework

Kluster allows you to process multibeam data and get fully corrected georeferenced soundings.  Here, let me show you.  It just so happens that I have a multibeam file here for us to look at.

In [ ]:
from HSTB.kluster import fqpr_convenience
multibeam_file = r'/home/jovyan/test_data/0009_20170523_181119_FA2806.all'

Note that Kluster currently only supports .all and .kmall files.

The multibeam file is a Kongsberg EM2040 that came off of one of the NOAA Ship Fairweather survey launches.  The fqpr_convenience is the Fully Qualified Ping Record Convenience module.  Think of this as the toolbox that has most of the tools you would need in the console.

Let's go ahead and do the all-in-one-step processing.  This will give us georeferenced soundings in a projected UTM zone with the waterline (as described in the .all file) as the vertical reference.

We are only passing in one file here, but you can also pass in a list of files or a directory of multibeam files to process.

In [ ]:
dataset = fqpr_convenience.perform_all_processing(multibeam_file)

Now dataset equals a new Fqpr instance, containing all of our converted and raw data.  It's all written to disk right alongside our multibeam file, which is what happens if you don't specify an output directory.

In [ ]:
dataset.multibeam.converted_pth

All of our converted and processed data is written to Zarr data stores, which you can open and interact with using Xarray objects and methods.  For instance, we can examine the raw attitude by doing:

In [ ]:
dataset.multibeam.raw_att

This is an Xarray Dataset containing the heading, heave, pitch, etc. that was in the .all file and is now available for you to analyze.  Let's plot the heave just to see how significant the heave for this line was.  Cool thing about xarray objects, is they have much of the numpy operations built in, as well as some rudimentary plotting functions.

In [ ]:
dataset.multibeam.raw_att.heave.plot()
print('Units of "{}"'.format(dataset.multibeam.raw_att.units['heave']))
print('Reference of "{}"'.format(dataset.multibeam.raw_att.reference['heave']))

Not too much heave here.  Note that the time here is UNIX time, in seconds.  So this little line would only be about 50 seconds long, going by the logged attitude.  How about navigation?  Our converted multibeam data should also have navigation in it.  Let's see if we can plot the altitude as well.

In [ ]:
dataset.multibeam.raw_nav.altitude.plot()
print('Units of "{}"'.format(dataset.multibeam.raw_nav.units['altitude']))
print('Reference of "{}"'.format(dataset.multibeam.raw_nav.reference['altitude']))

Both of those look pretty good.  How about pings?  That's what we are really after, the georeferenced xyz for each sounding.  Well, admittedly this gets a little more complicated with Kluster.  Kluster will take each serial number/sector/frequency combinateion and split them up into separate datasets.  This is to allow us to use interpolation and selection by time within Xarray, as Kongsberg multibeam files will have instances where pings with different sectors or frequencies or serial numbers (dual head systems) can be at the same time, and we need to have all unique times in each Xarray Dataset.

So let's look at the ping records that we have

In [ ]:
print([ping.system_identifier for ping in dataset.multibeam.raw_ping])

So you can see that unlike raw_att or raw_nav, raw_ping is actually a list of Xarray Dataset objects.  Here we have 1 dataset total, as this is not a dual head system.  A dual head system would have two datasets, one for each head.  Let's examine the dataset, which would be all the soundings that belong to sonar serial number 40111.

In [ ]:
dataset.multibeam.raw_ping[0]

You can click the little arrows next to 'Data Variables' or 'Attributes' to expand.  Yikes, that is a lot of information.  Our variables are all the converted, intermediate and final processed data records.  For instance 'beampointingangle' would be the raw beam angles from the .all file, while 'corr_pointing_angle' are the kluster corrected beam angles for attitude/mounting angle.

What we are after are the x, y, z data variables.  If you just want those values in text, we can just export to csv files, where you can import them into software packages like Caris HIPS or Qimera.  Let's try it out.

In [ ]:
dataset.export_pings_to_file()

You'll see there is a new 'csv_export' folder in /test_data/converted, that contains 6 new xyz files.  If you have access to Caris, Qimera or a software package that can ingest csv northing/easting/down files, try importing these, see what you get.  

We have six files total, as the default export setting is to separate the exported files by serialnumber_sector_frequency, as you can tell by the file names.

One thing that you will need is the coordinate system information for our georeferenced soundings.  Getting this is simple:

In [ ]:
print(type(dataset.xyz_crs))
print(dataset.xyz_crs)
print(dataset.xyz_crs.to_epsg())

You can see that we retain the pyproj CRS object as a class attribute, so you can easily use the existing methods to get EPSG or which ever format you want.

This line is roughly 90 meters deep, so I did an 8 meter surface which seemed to look pretty good.  It's a short line, so it doesn't really look all that interesting.

We can also plot this line in Kluster/matplotlib, using the plotting module.  I'll mention here that plotting this many soundings can be quite slow.

In [ ]:
fig = dataset.plot.soundings_plot_2d('georef', color_by='depth')

We can also plot with different colors or even plot just the sound velocity corrected offsets (alongtrack/acrosstrack/down)

In [ ]:
fig = dataset.plot.soundings_plot_2d('georef', color_by='sector')
fig = dataset.plot.soundings_plot_2d('svcorr', color_by='depth')

The svcorr plot looks a little odd, but note that the scale of the y axis (alongtrack) is quite restricted.

In [ ]:
fig = dataset.plot.soundings_plot_2d('svcorr', color_by='depth')
lims = fig.axes[0].set_ylim(-250,250)

Alternatively, we could just get the ping for the first time in our file:

In [ ]:
allpings = dataset.multibeam.raw_ping[0]
ping = dataset.multibeam.raw_ping[0].isel(time=0)

From here, we can do all kinds of analysis on the soundings.

In [ ]:
print('Mean depth: {}'.format(float(allpings.z.mean())))

In [ ]:
print('Mean depth for each ping:\n')
print(allpings.z.mean(axis=1))

In [ ]:
import matplotlib.pyplot as plt
print('all pings, alongtrack view, z positive down')
allpings.plot.scatter(x='y', y='z')
plt.gca().invert_yaxis()

In [ ]:
import matplotlib.pyplot as plt
print('first ping, alongtrack view')
ping.plot.scatter(x='y', y='z')
plt.gca().invert_yaxis()